## Load Data

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('lending-club-data.csv')
data.head()

/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1,1,1,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1,1,1,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1,1,1,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1,1,1,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1,1,1,0,5.21533,20141201T000000,1,1,1


In [3]:
data['safe_loans'] = data['bad_loans'].apply(lambda x : +1 if x==0 else -1)
data.drop('bad_loans', axis = 1, inplace = True)

## Subset Features

In [4]:
features_names = ['grade',              # grade of the loan
                  'term',               # the term of the loan
                  'home_ownership',     # home_ownership status: own, mortgage or rent
                  'emp_length',         # number of years of employment
                 ]
target_name = 'safe_loans'

In [5]:
features = data[features_names]
target = data[target_name]

In [6]:
def read_index(filename):
    with open(filename, 'r') as f:
        first_line = f.readline()
    first_line = first_line.translate(None,'[]"').strip().split(',')
    first_line = [x.strip() for x in first_line]
    return first_line

train_idx = read_index('train-idx.json')
validation_idx = read_index('valid-idx.json')
train_idx = [int(x) for x in train_idx]
validation_idx = [int(x) for x in validation_idx]

In [7]:
features = pd.get_dummies(features)
features[target_name] = target

In [8]:
train_data = features.iloc[train_idx]
validation_data = features.iloc[validation_idx]

In [9]:
train_data.reset_index(inplace = True)
validation_data.reset_index(inplace = True)
print features.columns.values

['grade_A' 'grade_B' 'grade_C' 'grade_D' 'grade_E' 'grade_F' 'grade_G'
 'term_ 36 months' 'term_ 60 months' 'home_ownership_MORTGAGE'
 'home_ownership_OTHER' 'home_ownership_OWN' 'home_ownership_RENT'
 'emp_length_1 year' 'emp_length_10+ years' 'emp_length_2 years'
 'emp_length_3 years' 'emp_length_4 years' 'emp_length_5 years'
 'emp_length_6 years' 'emp_length_7 years' 'emp_length_8 years'
 'emp_length_9 years' 'emp_length_< 1 year' 'emp_length_n/a' 'safe_loans']


## Weighted Decision Trees

In [130]:
def intermediate_node_weighted_mistakes(labels_in_node, data_weights):
    # Sum the weights of all entries with label +1
    total_weight_positive = sum(data_weights[labels_in_node == +1])
    
    # Weight of mistakes for predicting all -1's is equal to the sum above
    weighted_mistakes_all_negative = sum(data_weights[labels_in_node == +1])
    
    # Sum the weights of all entries with label -1
    total_weight_negative = sum(data_weights[labels_in_node == -1])
    
    # Weight of mistakes for predicting all +1's is equal to the sum above
    weighted_mistakes_all_positive = sum(data_weights[labels_in_node == -1])
    
    # Return the tuple (weight, class_label) representing the lower of the two weights
    #    class_label should be an integer of value +1 or -1.
    # If the two weights are identical, return (weighted_mistakes_all_positive,+1)
    if weighted_mistakes_all_negative > weighted_mistakes_all_positive:
        return (weighted_mistakes_all_positive, +1)
    elif weighted_mistakes_all_negative < weighted_mistakes_all_positive:
        return (weighted_mistakes_all_negative, -1)
    
    return (weighted_mistakes_all_positive, +1)

## Quiz Question: If we set the weights α=1 for all data points, how is the weight of mistakes WM(α,ŷ) related to the classification error?

It should be the same things

In [131]:
# If the data is identical in each feature, this function should return None
def best_splitting_feature(data, features, target, data_weights):
    
    # These variables will keep track of the best feature and the corresponding error
    best_feature = None
    best_error = float('+inf') 
    num_points = float(len(data))

    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        # The right split will have all data points where the feature value is 1
        left_split = data[data[feature] == 0]
        right_split = data[data[feature] == 1]
        
        # Apply the same filtering to data_weights to create left_data_weights, right_data_weights
        left_data_weights = data_weights[data[feature] == 0]
        right_data_weights = data_weights[data[feature] == 1]    
        
        # Calculate the weight of mistakes for left and right sides
        left_weighted_mistakes, left_class = intermediate_node_weighted_mistakes(left_split[target],
                                                                                 left_data_weights)
        right_weighted_mistakes, right_class = intermediate_node_weighted_mistakes(right_split[target],
                                                                                   right_data_weights)
        
        # Compute weighted error by computing
        #  ( [weight of mistakes (left)] + [weight of mistakes (right)] ) / [total weight of all data points]
        error = (left_weighted_mistakes + right_weighted_mistakes)/sum(data_weights)
        # If this is the best error we have found so far, store the feature and the error
        
        if error < best_error:
            print '=============='
            print left_weighted_mistakes, right_weighted_mistakes, sum(data_weights)
            print feature, error
            best_feature = feature
            best_error = error
    
    # Return the best feature we found
    return best_feature

## Create leaf function

In [132]:
def create_leaf(target_values, data_weights):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'is_leaf': True}
    
    # Computed weight of mistakes.
    # Store the predicted class (1 or -1) in leaf['prediction']
    weighted_error, best_class = intermediate_node_weighted_mistakes(target_values, data_weights)
    leaf['prediction'] = best_class
    
    return leaf

## Weighted Decision Tree Create Process

In [133]:
def weighted_decision_tree_create(data, features, target, data_weights, current_depth = 1, max_depth = 10):
    print "---- START -----"
    remaining_features = features[:] # Make a copy of the features.
    target_values = data[target]
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    
    # Stopping condition 1. Error is 0.
    if intermediate_node_weighted_mistakes(target_values, data_weights)[0] <= 1e-15:
        print "Stopping condition 1 reached."                
        return create_leaf(target_values, data_weights)
    
    # Stopping condition 2. No more features.
    if remaining_features == []:
        print "Stopping condition 2 reached."                
        return create_leaf(target_values, data_weights)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth > max_depth:
        print "Reached maximum depth. Stopping for now."
        return create_leaf(target_values, data_weights)
    
    # If all the datapoints are the same, splitting_feature will be None. Create a leaf
    splitting_feature = best_splitting_feature(data, features, target, data_weights)
    remaining_features.remove(splitting_feature)
        
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    left_data_weights = data_weights[data[splitting_feature] == 0]
    right_data_weights = data_weights[data[splitting_feature] == 1]
    
    print "Split on feature %s. (%s, %s)" % (\
              splitting_feature, len(left_split), len(right_split))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print "Creating leaf node."
        return create_leaf(left_split[target], data_weights)
    if len(right_split) == len(data):
        print "Creating leaf node."
        return create_leaf(right_split[target], data_weights)
    
    # Repeat (recurse) on left and right subtrees
    left_tree = weighted_decision_tree_create(
        left_split, remaining_features, target, left_data_weights, current_depth + 1, max_depth)
    right_tree = weighted_decision_tree_create(
        right_split, remaining_features, target, right_data_weights, current_depth + 1, max_depth)
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

## Count nodes aka tree complexity

In [134]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

## Classify fun

In [135]:
def classify(tree, x, annotate = False):   
    # If the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction'] 
    else:
        # Split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

## Evaluating the tree

In [136]:
def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    prediction = []
    for i in range(data.shape[0]):
        prediction.append(classify(tree, data.iloc[i]))
    
    errors = np.sum(prediction != data[target_name])
    return errors / float(len(prediction))

## Intuition begind weights

In [137]:
# Assign weights
example_data_weights = np.array([1.] * 10 + [0.] * (len(train_data) - 20) + [1.] * 10)
print(example_data_weights[:15])
print list(features.columns.values[:-1])

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.]
['grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G', 'term_ 36 months', 'term_ 60 months', 'home_ownership_MORTGAGE', 'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT', 'emp_length_1 year', 'emp_length_10+ years', 'emp_length_2 years', 'emp_length_3 years', 'emp_length_4 years', 'emp_length_5 years', 'emp_length_6 years', 'emp_length_7 years', 'emp_length_8 years', 'emp_length_9 years', 'emp_length_< 1 year', 'emp_length_n/a']


In [138]:
# Train a weighted decision tree model.
small_data_decision_tree_subset_20 = weighted_decision_tree_create(train_data,  
                                                                   list(features.columns.values[:-1]), 'safe_loans',
                                                                   example_data_weights, max_depth=2)

---- START -----
--------------------------------------------------------------------
Subtree, depth = 1 (37219 data points).

/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  app.launch_new_instance()
/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:9: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:12: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:18: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__


31115.0 6104.0 20.0
grade_A 1860.95
Split on feature grade_A. (31115, 6104)
---- START -----
--------------------------------------------------------------------
Subtree, depth = 2 (31115 data points).
19937.0 11178.0 37219.0
grade_B 0.835997743088
Split on feature grade_B. (19937, 11178)
---- START -----
--------------------------------------------------------------------
Subtree, depth = 3 (19937 data points).
Reached maximum depth. Stopping for now.
---- START -----
--------------------------------------------------------------------
Subtree, depth = 3 (11178 data points).
Reached maximum depth. Stopping for now.
---- START -----
--------------------------------------------------------------------
Subtree, depth = 2 (6104 data points).
6104.0 0 37219.0
grade_B 0.164002256912
Split on feature grade_B. (6104, 0)
Creating leaf node.


/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:31: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index


In [127]:
evaluate_classification_error(small_data_decision_tree_subset_20, train_data)

0.27260270292055133

## Quiz Question: Will you get the same model as small_data_decision_tree_subset_20 if you trained a decision tree with only the 20 data points with non-zero weights from the set of points in subset_20?



Yes

## Implementing AdaBoost

In [128]:
from math import log
from math import exp

def adaboost_with_tree_stumps(data, features, target, num_tree_stumps):
    # start with unweighted data
    alpha = np.array([1./len(data)]*len(data))
    weights = []
    tree_stumps = []
    target_values = data[target]
    
    for t in xrange(num_tree_stumps):
        print '====================================================='
        print 'Adaboost Iteration %d' % t
        print '====================================================='        
        # Learn a weighted decision tree stump. Use max_depth=1
        tree_stump = weighted_decision_tree_create(data, features, target, data_weights=alpha, max_depth=1)
        tree_stumps.append(tree_stump)
        
        # Make predictions
        predictions = []
        for i in range(data.shape[0]):
            predictions.append(classify(tree_stump, data.iloc[i]))
   
        # Produce a Boolean array indicating whether
        # each data point was correctly classified
        is_correct = predictions == target_values
        is_wrong   = predictions != target_values
        
        # Compute weighted error
        weighted_error = np.dot(is_wrong, alpha) / sum(alpha)
        # Compute model coefficient using weighted error
        weight = 0.5 * log((1 - weighted_error) / weighted_error)
        weights.append(weight)
        # Adjust weights on data point
        adjustment = map(lambda s : exp(-weight) if s else exp(weight), is_correct)
        # Scale alpha by multiplying by adjustment
        # Then normalize data points weights
        alpha = np.multiply(alpha, adjustment)
        alpha = alpha / np.sum(alpha)
    
    return weights, tree_stumps

## Train a Boosted Ensemble

In [129]:
adaboost_with_tree_stumps2 = adaboost_with_tree_stumps(train_data, list(features.columns.values[:-1]),
                                                      'safe_loans', num_tree_stumps = 10)

/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  app.launch_new_instance()
/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:9: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:12: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:18: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__

Adaboost Iteration 0
---- START -----
--------------------------------------------------------------------
Subtree, depth = 1 (37219 data points).
0.835997743089 0.164002256912
grade_A 1.0
Split on feature grade_A. (31115, 6104)
---- START -----
--------------------------------------------------------------------
Subtree, depth = 2 (31115 data points).
Reached maximum depth. Stopping for now.
---- START -----
--------------------------------------------------------------------
Subtree, depth = 2 (6104 data points).
Reached maximum depth. Stopping for now.
Adaboost Iteration 1
---- START -----
--------------------------------------------------------------------
Subtree, depth = 1 (37219 data points).
1.53336290164 0.300808200276
grade_A 1.83417110191
1.81633155924 0.0178395426769
grade_G 1.83417110191
1.83111571063 0.00305539128721
home_ownership_OTHER 1.83417110191
Split on feature home_ownership_OTHER. (37157, 62)
---- START -----
------------------------------------------------------

/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:30: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:31: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index


KeyboardInterrupt: 

## Evaluating AdaBoost Classifier

In [81]:
def predict_stump(tree_stump, data, stump_weight):
    predictions = []
    for i in range(data.shape[0]):
        predictions.append(classify(tree_stump, data.iloc[i]) * stump_weight)
    
    return predictions

def predict_adaboost(stump_weights, tree_stumps, data):
    scores = np.array([0.] * len(data))
    
    for i, tree_stump in enumerate(tree_stumps):
        predictions = predict_stump(tree_stump, data, stump_weights[i])
        scores = np.add(predictions, scores)
    
    return map(lambda score : +1 if score > 0 else -1, scores)

## Quiz Question: Are the weights monotonically decreasing, monotonically increasing, or neither?



In [82]:
print adaboost_with_tree_stumps2[0]

[0.49072869106598693, 1.7430501486611918e-13, 1.1768364061025273e-13, -1.1102230246251688e-14, -5.995204332975882e-15, -4.8849813083507124e-15, 1.176836406102652e-14, -5.995204332975882e-15, -4.8849813083507124e-15, 1.176836406102652e-14]


Monotonically decreasing

## AdaBoost with 30 estimators

In [83]:
adaboost30_stump = adaboost_with_tree_stumps(train_data, list(features.columns.values[:-1]), 'safe_loans', num_tree_stumps = 30)

/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  app.launch_new_instance()
/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:9: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:12: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:18: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__

Adaboost Iteration 0
---- START -----
--------------------------------------------------------------------
Subtree, depth = 1 (37219 data points).
Split on feature grade_A. (31115, 6104)
---- START -----
--------------------------------------------------------------------
Subtree, depth = 2 (31115 data points).
Reached maximum depth. Stopping for now.
---- START -----
--------------------------------------------------------------------
Subtree, depth = 2 (6104 data points).
Reached maximum depth. Stopping for now.
Adaboost Iteration 1
---- START -----
--------------------------------------------------------------------
Subtree, depth = 1 (37219 data points).
Split on feature home_ownership_OTHER. (37157, 62)
---- START -----
--------------------------------------------------------------------
Subtree, depth = 2 (37157 data points).
Reached maximum depth. Stopping for now.
---- START -----
--------------------------------------------------------------------
Subtree, depth = 2 (62 data p

/Users/oleksiirenov/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:31: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index


In [84]:
stump_weights, tree_stumps = adaboost30_stump

In [85]:
error_all = []
for n in xrange(1, 31):
    predictions = predict_adaboost(stump_weights[:n], tree_stumps[:n], train_data)
    error = 1.0 - np.sum(train_data['safe_loans'] == predictions)/float(len(predictions))
    error_all.append(error)
    print "Iteration %s, training error = %s" % (n, error_all[n-1])

Iteration 1, training error = 0.272602702921
Iteration 2, training error = 0.272602702921
Iteration 3, training error = 0.272602702921
Iteration 4, training error = 0.272602702921


KeyboardInterrupt: 

##Visualizing training error vs number of iterations

